# load packages

In [ ]:
import pandas as pd

In [ ]:
from datetime import datetime

In [ ]:
from zoneinfo import ZoneInfo

In [ ]:
import numpy as np

# read in input files

In [ ]:
icd = pd.read_csv('Phenotypes/3.0/PMBB-Release-2024-3.0_phenotype_condition_occurrence.txt',
                  sep = '\t',
                  usecols = ['person_id', 'condition_start_date', 'condition_source_value'])
icd.head()

In [ ]:
sex = pd.read_csv('Phenotypes/3.0/PMBB-Release-2024-3.0_covariates.txt',
                  sep = '\t',
                  usecols = ['person_id', 'Sequenced_gender', 'Batch'])
sex.head()

In [ ]:
dob = pd.read_csv('Phenotypes/3.0/PMBB-Release-2024-3.0_phenotype_person.txt',
                  sep = '\t',
                  usecols = ['person_id', 'birth_datetime'])
dob.head()

In [ ]:
egfr = pd.read_csv('Phenotypes/3.0/PMBB-Release-2024-3.0_phenotype_labs_egfr.txt',
                    sep = '\t')
egfr.head()

In [ ]:
egfr_calc = pd.read_csv('Phenotypes/3.0/PMBB-Release-2024-3.0_phenotype_labs_egfr_calculated.txt',
                        sep = '\t')
egfr_calc.head()

In [ ]:
death = pd.read_csv('Phenotypes/3.0/PMBB-Release-2024-3.0_phenotype_death.txt',
                    sep = '\t',
                    usecols = ['person_id', 'death_date'])
death.head()

In [ ]:
alb = pd.read_csv('Phenotypes/3.0/PMBB-Release-2024-3.0_phenotype_labs_albumin.txt',
                   sep = '\t')
alb.head()

In [ ]:
cre = pd.read_csv('Phenotypes/3.0/PMBB-Release-2024-3.0_phenotype_labs_creatinine_urine.txt',
                  sep = '\t')
cre.head()

In [ ]:
cpt = pd.read_csv('Phenotypes/3.0/PMBB-Release-2024-3.0_phenotype_procedure_occurrence.txt',
                  sep = '\t')
cpt.head()

# identify cases and controls

In [ ]:
phecode.value_counts('GU_582.2')

In [ ]:
case = phecode[phecode['GU_582.2'] == True]
case.value_counts('GU_582.2')

In [ ]:
control = phecode[phecode['GU_582.2'] == False]
control.value_counts('GU_582.2')

In [ ]:
alb_cre = alb_urine_sub.merge(cre_urine_sub, on = 'PMBB_ID', how = 'inner')
print(len(alb_urine_sub.index))
print(len(cre_urine_sub.index))
print(len(alb_cre.index))

In [ ]:
egfr_cre = egfr_mean.merge(cre_serum_mean, on = 'PMBB_ID', how = 'inner')
print(len(egfr_mean.index))
print(len(cre_serum_mean.index))
print(len(egfr_cre.index))

In [ ]:
transplant_icd = ['V42.0', 'Z94.0', 'Y60.2', 'Y61.2', 'Y84.1']

In [ ]:
transplant_indiv = icd[icd['code'].isin(transplant_icd)]
print(len(transplant_indiv['PMBB_ID'].unique()))
transplant_indiv['code'].unique()

In [ ]:
icd_no_transplant = icd_cpt_ckd[~icd_cpt_ckd['PMBB_ID'].isin(transplant_indiv['PMBB_ID'])]
print(len(icd_no_transplant['PMBB_ID'].unique()))

In [ ]:
icd_cpt_no_transplant = icd_no_transplant[~icd_no_transplant['PMBB_ID'].isin(cpt_kt['PMBB_ID'])]
print(len(icd_cpt_no_transplant['PMBB_ID'].unique()))

In [ ]:
aki_indiv = icd_ckd[icd_ckd['code'].str.contains('|'.join(aki_icd))]
aki_indiv = aki_indiv[~aki_indiv['PMBB_ID'].isin(transplant_indiv['PMBB_ID'])]
len(aki_indiv['PMBB_ID'].unique())

In [ ]:
id_list = aki_indiv['PMBB_ID'].unique()
len(id_list)

In [ ]:
not_only_aki = []
for id in id_list:
    df = aki_indiv[aki_indiv['PMBB_ID'].isin([id])]
    code = df[~df['code'].isin(aki_icd)]
    if len(code.index) > 0:
        not_only_aki.append(id)

In [ ]:
len(not_only_aki)

In [ ]:
not_only_aki_df = aki_indiv[aki_indiv['PMBB_ID'].isin(not_only_aki)]
len(not_only_aki_df['PMBB_ID'].unique())

In [ ]:
no_aki = icd_cpt_no_transplant[~icd_cpt_no_transplant['PMBB_ID'].isin(aki_indiv['PMBB_ID'])]
len(no_aki['PMBB_ID'].unique())

In [ ]:
icd_aki_fixed = pd.concat([no_aki, not_only_aki_df], axis = 0)
print(len(icd_aki_fixed['PMBB_ID'].unique()))
icd_aki_fixed.head()

In [ ]:
id_list = icd_aki_fixed['PMBB_ID'].unique()
len(id_list)

In [ ]:
two_instances = []
for id in id_list:
    id_df = icd_aki_fixed[icd_aki_fixed['PMBB_ID'].isin([id])]
    id_df.drop_duplicates(inplace = True)
    id_df.reset_index(inplace = True, drop = True)
    if len(id_df.index) >= 2:
        two_instances.append(id)

In [ ]:
len(two_instances)

In [ ]:
icd_two = icd_aki_fixed[icd_aki_fixed['PMBB_ID'].isin(two_instances)]
print(len(icd_two['PMBB_ID'].unique()))
icd_two.head()

In [ ]:
icd_egfr = icd_two.merge(egfr_mean, on = 'PMBB_ID', how = 'inner')
print(len(icd_egfr.index))
icd_egfr.head()

In [ ]:
case = icd_egfr[icd_egfr['eGFR'] < 60]
print(len(case.index))

In [ ]:
case['CKD'] = 1
case.drop(columns = ['code', 'condition_start_date', 'RESULT_DATE_SHIFT', 'eGFR'], inplace = True)
case.head()

## v3

### clean labs

#### egfr measured

In [ ]:
egfr_sub = egfr[['person_id', 'measurement_datetime', 'value_converted']]
egfr_sub['measurement_datetime'] = pd.to_datetime(egfr_sub['measurement_datetime'])
egfr_sub.rename(columns = {'value_converted' : 'eGFR'}, inplace = True)
egfr_mean = egfr_sub.groupby('person_id', as_index = False).mean()
print(len(egfr_mean.index))
print(len(egfr_mean['person_id'].unique()))
print(egfr_mean['eGFR'].describe().map(lambda x: f"{x:.6f}"))
print(len(egfr_mean[egfr_mean['eGFR'] < 60].index))
egfr_mean.head()

#### eGFR calculated

In [ ]:
egfr_calc_sub = egfr_calc[['person_id', 'measurement_datetime', 'value_converted']]
egfr_calc_sub['measurement_datetime'] = pd.to_datetime(egfr_calc_sub['measurement_datetime'])
egfr_calc_sub.rename(columns = {'value_converted' : 'eGFR'}, inplace = True)
egfr_calc_mean = egfr_calc_sub.groupby('person_id', as_index = False).mean()
print(len(egfr_calc_mean.index))
print(len(egfr_calc_mean['person_id'].unique()))
print(egfr_calc_mean['eGFR'].describe().map(lambda x: f"{x:.6f}"))
print(len(egfr_calc_mean[egfr_calc_mean['eGFR'] < 60].index))
egfr_calc_mean.head()

#### combine eGFR dataframes

In [ ]:
egfr_all = pd.concat([egfr_mean, egfr_calc_mean], axis = 0)
egfr_all_mean = egfr_all.groupby('person_id', as_index = False).mean()
print(len(egfr_all_mean.index))
print(len(egfr_all_mean['person_id'].unique()))
print(len(egfr_all_mean[egfr_all_mean['eGFR'] < 60].index))

### clean ICD and CPT dfs

#### filter to CPT codes and convert to numeric

#### subset, rename, and convert datetime in CPT

In [ ]:
cpt_sub = cpt[['person_id', 'procedure_datetime', 'procedure_source_value']]
cpt_sub.rename(columns = {'procedure_datetime' : 'date',
                         'procedure_source_value' : 'code'}, inplace = True)
cpt_sub['date'] = pd.to_datetime(cpt_sub['date'])
cpt_sub['code'] = pd.to_numeric(cpt_sub['code'], errors = 'coerce')
cpt_sub.dropna(subset = ['code'], inplace = True)
cpt_sub['code'] = cpt_sub['code'].astype(int)
cpt_sub.head()

#### filter CPT to ESRD and KT codes

In [ ]:
cpt_esrd = cpt_sub[((cpt_sub['code'] >= 90935) & (cpt_sub['code'] <= 90999)) |
                ((cpt_sub['code'] >= 36800) & (cpt_sub['code'] <= 36821)) |
                ((cpt_sub['code'] >= 49418) & (cpt_sub['code'] <= 49422)) |
                ((cpt_sub['code'] >= 50220) & (cpt_sub['code'] <= 50240))]
print(len(cpt_esrd['person_id'].unique()))
cpt_esrd['code'].value_counts(dropna = False)

In [ ]:
cpt_kt = cpt_sub[((cpt_sub['code'] >= 50300) & (cpt_sub['code'] <= 50380))]
print(len(cpt_kt['person_id'].unique()))
cpt_kt['code'].value_counts(dropna = False)

#### rename ICD file

In [ ]:
icd.rename(columns = {'condition_start_date' : 'date',
                      'condition_source_value' : 'code'}, inplace = True)

#### filter to CKD ICD codes

In [ ]:
ckd_icd = ['N18',
           'N19',
           'N03',
           'N07',
           'N11',
           'Q60',
           'Q61',
           'Q63',
           '582.9',
           '585.9',
           '586.9',
           '587.9',
           '588.84',
           '588.9',
           '581.9',
           '583.02',
           '583.08',
           '583.09',
           '583.1',
           '583.29',
           '583.8',
           '583.9',
           '588.4',
           '589.1',
           'E10.2',
           'E11.2',
           'E13.2',
           'E14.2',
           'I12.9',
           'I13',
           'N00',
           'N01',
           'N02',
           'N04',
           'N05',
           'N06',
           'N08',
           'N10',
           'N12',
           'N13',
           'N14',
           'N15',
           'N16',
           'N27.1',
           'V45.1',
           'V56.8',
           'E85.3',
           'N16.5',
           'N18.0',
           'N18.5',
           'Q60.1',
           'T82.4',
           'T86.1',
           'Z49',
           'Z99.2']

In [ ]:
aki_icd = ['584.5', '584.9', 'N17']

In [ ]:
all_ckd_icd = ckd_icd + aki_icd

In [ ]:
icd_ckd = icd[icd['code'].str.contains('|'.join(all_ckd_icd))]
print(len(icd_ckd['person_id'].unique()))
icd_ckd['code'].unique()

#### concatenate ICD and CPT dfs

In [ ]:
icd_cpt_ckd = pd.concat([icd_ckd, cpt_esrd], axis = 0)
print(len(icd_cpt_ckd['person_id'].unique()))

#### remove individuals with kidney transplant

In [ ]:
transplant_icd = ['V42.0', 'Z94.0', 'Y60.2', 'Y61.2', 'Y84.1']

In [ ]:
transplant_indiv = icd[icd['code'].isin(transplant_icd)]
print(len(transplant_indiv['person_id'].unique()))
transplant_indiv['code'].unique()

In [ ]:
icd_no_transplant = icd_cpt_ckd[~icd_cpt_ckd['person_id'].isin(transplant_indiv['person_id'])]
print(len(icd_no_transplant['person_id'].unique()))

In [ ]:
icd_cpt_no_transplant = icd_no_transplant[~icd_no_transplant['person_id'].isin(cpt_kt['person_id'])]
print(len(icd_cpt_no_transplant['person_id'].unique()))

#### remove individuals with only acute kidney injury

In [ ]:
aki_indiv = icd_ckd[icd_ckd['code'].str.contains('|'.join(aki_icd))]
aki_indiv = aki_indiv[~aki_indiv['person_id'].isin(transplant_indiv['person_id'])]
len(aki_indiv['person_id'].unique())

In [ ]:
id_list = aki_indiv['person_id'].unique()
len(id_list)

In [ ]:
not_only_aki = []
for id in id_list:
    df = aki_indiv[aki_indiv['person_id'].isin([id])]
    code = df[~df['code'].isin(aki_icd)]
    if len(code.index) > 0:
        not_only_aki.append(id)

In [ ]:
len(not_only_aki)

In [ ]:
not_only_aki_df = aki_indiv[aki_indiv['person_id'].isin(not_only_aki)]
len(not_only_aki_df['person_id'].unique())

In [ ]:
no_aki = icd_cpt_no_transplant[~icd_cpt_no_transplant['person_id'].isin(aki_indiv['person_id'])]
len(no_aki['person_id'].unique())

In [ ]:
icd_aki_fixed = pd.concat([no_aki, not_only_aki_df], axis = 0)
print(len(icd_aki_fixed['person_id'].unique()))
icd_aki_fixed.head()

#### apply rule of 2

In [ ]:
id_list = icd_aki_fixed['person_id'].unique()
len(id_list)

In [ ]:
two_instances = []
for id in id_list:
    id_df = icd_aki_fixed[icd_aki_fixed['person_id'].isin([id])]
    id_df.drop_duplicates(inplace = True)
    id_df.reset_index(inplace = True, drop = True)
    if len(id_df.index) >= 2:
        two_instances.append(id)

In [ ]:
len(two_instances)

In [ ]:
icd_two = icd_aki_fixed[icd_aki_fixed['person_id'].isin(two_instances)]
print(len(icd_two['person_id'].unique()))
icd_two.head()

#### filter to earliest date

In [ ]:
icd_two.sort_values(by = ['person_id', 'date'], ascending = True, inplace = True)
icd_two.drop_duplicates(subset = 'person_id', keep = 'first', inplace = True)
print(len(icd_two.index))
icd_two.head()

### merge eGFR and ICD

In [ ]:
icd_egfr = icd_two.merge(egfr_all_mean, on = 'person_id', how = 'inner')
print(len(icd_egfr.index))
icd_egfr.head()

### filter to people with eGFR < 60

In [ ]:
case = icd_egfr[icd_egfr['eGFR'] < 60]
print(len(case.index))

### get first date

In [ ]:
case['date'] = pd.to_datetime(case['date'], utc=True).dt.tz_localize(None)
case['measurement_datetime'] = pd.to_datetime(case['measurement_datetime'], utc=True).dt.tz_localize(None)
case['first_date'] = case[['date', 'measurement_datetime']].min(axis = 1)
case.head()

## add CKD column and subset

In [ ]:
case['CKD'] = 1
case.drop(columns = ['code', 'date', 'measurement_datetime', 'eGFR'], inplace = True)
case.head()

# clean demo df

## v3

### merge dfs

In [ ]:
demo = sex.merge(dob, on = 'person_id', how = 'inner')
print(len(demo.index))
print(len(demo['person_id'].unique()))
demo.head()

### recode sex column

In [ ]:
demo['Sequenced_gender'].unique()

In [ ]:
demo['Sequenced_gender'] = demo['Sequenced_gender'].str.replace('Male','1')
demo['Sequenced_gender'] = demo['Sequenced_gender'].str.replace('Female','2')
demo['Sequenced_gender'].unique()

### convert dob column to date type

In [ ]:
demo['birth_datetime'] = pd.to_datetime(demo['birth_datetime'])

# calculate date at last data release for controls

### filter to controls

In [ ]:
print(len(demo_sex.index))
demo_control = demo_sex[demo_sex['PMBB_ID'].isin(egfr_mean['PMBB_ID'])]
print(len(demo_control.index))
demo_control = demo_control[~demo_control['PMBB_ID'].isin(icd_cpt_ckd['PMBB_ID'])]
print(len(demo_control.index))
demo_control = demo_control[~demo_control['PMBB_ID'].isin(transplant_indiv['PMBB_ID'])]
print(len(demo_control.index))
demo_control = demo_control[~demo_control['PMBB_ID'].isin(cpt_kt['PMBB_ID'])]
len(demo_control.index)

### calculate age for individuals who are dead

In [ ]:
dead = demo_control[demo_control['death_date_shift'].isna() == False]
print(len(dead.index))
print(dead['death_date_shift'].max())
dead.head()

In [ ]:
dead['AGE'] = pd.to_datetime(dead['death_date_shift']) - pd.to_datetime(dead['birth_date_shift'])
dead['AGE'] = dead['AGE'].astype(str).str.replace(' days','')
dead['AGE'] = dead['AGE'].astype(float)
dead['AGE'] = dead['AGE']/365.2425
dead.head()

### calculate age for people who are alive

In [ ]:
alive = demo_control[demo_control['death_date_shift'].isna() == True]
print(len(alive.index))
print(alive['death_date_shift'].max())
alive.head()

In [ ]:
alive['AGE'] = (datetime(2023, 6, 13, 0, 0, 0)) - pd.to_datetime(alive['birth_date_shift'])
alive['AGE'] = alive['AGE'].astype(str).str.replace(' days','')
alive['AGE'] = alive['AGE'].astype(float)
alive['AGE'] = alive['AGE']/365.2425
alive.head()

### concatenate

In [ ]:
demo_control_age = pd.concat([alive, dead],axis = 0)
print(len(demo_control_age.index))
demo_control_age.head()

### add CKD column

In [ ]:
demo_control_age['CKD'] = 0

### subset

In [ ]:
demo_control_age.drop(columns = ['birth_date_shift','death_date_shift'], inplace = True)
demo_control_age.head()

## v3

### filter to controls

In [ ]:
print(len(demo.index))
demo_control = demo[demo['person_id'].isin(egfr_all_mean['person_id'])]
print(len(demo_control.index))
demo_control = demo_control[~demo_control['person_id'].isin(icd_cpt_ckd['person_id'])]
print(len(demo_control.index))
demo_control = demo_control[~demo_control['person_id'].isin(transplant_indiv['person_id'])]
print(len(demo_control.index))
demo_control = demo_control[~demo_control['person_id'].isin(cpt_kt['person_id'])]
len(demo_control.index)

### calculate age for people who are dead

In [ ]:
dead = demo_control.merge(death, on = 'person_id', how = 'inner')
print(len(death.index))
print(len(dead.index))
dead.head()

In [ ]:
dead['AGE'] = pd.to_datetime(dead['death_date']) - dead['birth_datetime']
dead['AGE'] = dead['AGE'].astype(str).str.replace(' days','')
dead['AGE'] = dead['AGE'].astype(float)
dead['AGE'] = dead['AGE']/365.2425
dead.head()

In [ ]:
dead.drop(columns = ['birth_datetime', 'death_date'], inplace = True)

### calculate age for people who are alive

In [ ]:
alive = demo_control[~demo_control['person_id'].isin(death['person_id'])]
print(len(demo_control.index))
print(len(alive.index))
alive.head()

In [ ]:
alive['AGE'] = pd.to_datetime('2025-01-30') - alive['birth_datetime']
alive['AGE'] = alive['AGE'].astype(str).str.replace(' days','')
alive['AGE'] = alive['AGE'].astype(float)
alive['AGE'] = alive['AGE']/365.2425
alive.head()

In [ ]:
alive.drop(columns = ['birth_datetime'], inplace = True)
alive.head()

### concatenate

In [ ]:
demo_control_age = pd.concat([alive, dead],axis = 0)
print(len(demo_control.index))
print(len(demo_control_age.index))
demo_control_age.head()

### add ckd column

In [ ]:
demo_control_age['CKD'] = 0

# calculate age at first condition for cases

### original

#### filter to cases

In [ ]:
demo_case = demo_sex[demo_sex['PMBB_ID'].isin(case['id'])]
print(len(demo_sex.index))
print(len(demo_case['PMBB_ID'].unique()))
len(demo_case.index)

#### subset icd column to phecodeX codes

In [ ]:
phecode_map_ckd = phecode_map[phecode_map['phecode'].isin(['GU_582.2'])]
print(phecode_map_ckd['phecode'].unique())
print(phecode_map_ckd['ICD'].unique())
phecode_map_ckd.head()

In [ ]:
icd.head()

In [ ]:
icd_case = icd[icd['code'].isin(phecode_map_ckd['ICD'])]
print(icd_case['code'].unique())

#### get earliest date

In [ ]:
icd_case['condition_start_date'] = pd.to_datetime(icd_case['condition_start_date'])

In [ ]:
icd_case.sort_values(by=['PMBB_ID','condition_start_date'],inplace=True)

In [ ]:
icd_case_first = icd_case.drop_duplicates(keep='first',subset='PMBB_ID')
print(len(icd_case_first.index))
print(len(icd_case_first['PMBB_ID'].unique()))
icd_case_first.head()

In [ ]:
icd_case_first.drop(columns = ['code'], inplace = True)

#### merge icd and demo df

In [ ]:
demo_case_icd = demo_case.merge(icd_case_first, on = 'PMBB_ID')
print(len(demo_case_icd.index))
demo_case_icd.head()

#### calculate age

In [ ]:
demo_case_icd['AGE'] = demo_case_icd['condition_start_date'] - pd.to_datetime(demo_case_icd['birth_date_shift'])
demo_case_icd['AGE'] = demo_case_icd['AGE'].astype(str).str.replace(' days','')
demo_case_icd['AGE'] = demo_case_icd['AGE'].astype(float)
demo_case_icd['AGE'] = demo_case_icd['AGE']/365.2425
demo_case_icd.head()

#### add CKD column

In [ ]:
demo_case_icd['CKD'] = 1

#### subset

In [ ]:
demo_case_icd.drop(columns = ['birth_date_shift','death_date_shift','condition_start_date'], inplace = True)
demo_case_icd.head()

### YK phenotyping

#### merge demo and icd

In [ ]:
demo_case = demo_sex.merge(case, on = 'person_id', how = 'inner')
print(len(demo_sex.index))
print(len(demo_case['person_id'].unique()))
print(len(demo_case.index))
demo_case.head()

#### calculate age

In [ ]:
demo_case['AGE'] = demo_case['first_date'] - pd.to_datetime(demo_case['birth_date_shift'])
demo_case['AGE'] = demo_case['AGE'].astype(str).str.replace(r' days .*', '', regex = True)
demo_case['AGE'] = demo_case['AGE'].astype(float)
demo_case['AGE'] = demo_case['AGE']/365.2425
demo_case.head()

#### reorder columns

In [ ]:
demo_case_sub = demo_case[['PMBB_ID', 'gender', 'AGE', 'CKD']]
demo_case_sub.head()

## v3

### merge case and demo dfs

In [ ]:
demo_case = case.merge(demo, on = 'person_id', how = 'inner')
print(len(case['person_id'].unique()))
print(len(demo_case['person_id'].unique()))
demo_case.head()

### calculate age

In [ ]:
demo_case['AGE'] = demo_case['first_date'] - demo_case['birth_datetime']
demo_case['AGE'] = demo_case['AGE'].astype(str).str.replace(r' days.*', '', regex = True)
demo_case['AGE'] = demo_case['AGE'].astype(float)
demo_case['AGE'] = demo_case['AGE']/365.2425
demo_case.head()

### subset

In [ ]:
demo_case.drop(columns = ['first_date', 'birth_datetime'], inplace = True)
demo_case.head()

### add CKD column

In [ ]:
demo_case['CKD'] = 1

# concatenate case and control dfs

### original

In [ ]:
case_control = pd.concat([demo_case_icd, demo_control_age], axis = 0)
print(len(case_control.index))
case_control.head()

### YK phenotyping

In [ ]:
case_control = pd.concat([demo_case_sub, demo_control_age], axis = 0)
print(len(case_control.index))
print(case_control['CKD'].value_counts(dropna = False))
case_control.head()

## v3

In [ ]:
case_control = pd.concat([demo_case, demo_control_age], axis = 0)
print(len(case_control.index))
case_control.head()

# rename sex and batch columns

In [ ]:
case_control.rename(columns = {'gender' : 'SEX'}, inplace = True)
case_control.head()

## v3

In [ ]:
case_control.rename(columns = {'Sequenced_gender' : 'SEX',
                              'Batch' : 'BATCH'}, inplace = True)
case_control.head()

In [ ]:
case_control['BATCH'].value_counts()

# export

### original

In [ ]:
case_control.to_csv('/project/ritchie/projects/PGS_variability/CKD/PMBB/input/PMBB_v2.CKD_PGS.phenotype_covariates.txt',
                    sep = '\t',
                    index = None)

### updated phenotyping

In [ ]:
case_control.to_csv('/project/ritchie/projects/PGS_variability/CKD/PMBB/input/PMBB_v2.CKD_PGS.YK_phenotyping.phenotype_covariates.txt',
                    sep = '\t',
                    index = None)

## v3

In [ ]:
case_control.to_csv('input/PMBB_v3.CKD_PGS.YK_phenotyping.phenotype_covariates.txt',
                    sep = '\t',
                    index = None)